In [ ]:
# 전처리 동안 사용할 라이브러리 종합
import pandas as pd
import numpy as np

import re
import copy
import datetime
from tqdm import tqdm

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.family'] = 'AppleGothic'
sns.set(font="AppleGothic", 
        rc={"axes.unicode_minus":False},
        style='darkgrid')

import warnings
warnings.filterwarnings('ignore')

### 주식 가격데이터 전처리

In [ ]:
# 주가 데이터 로드 및 통합

# 주가데이터 파일 통합
import glob
import os
from tqdm import tqdm

file_path = glob.glob("../data/raw_data/stock_code/*.csv")

stock_price = pd.DataFrame()
for f in tqdm(file_path):
    code = os.path.basename(f).split(".")[0]
    df_temp = pd.read_csv(f, encoding='cp949', index_col=0)
    df_temp["종목코드"] = code
    stock_price = pd.concat([stock_price, df_temp])

# 사용할 컬럼 선정
use_cols = ["날짜","시가","고가","저가","종가","전일비","등락률","거래량",'금액(백만)'\
    , '신용비', "개인", "기관", "외인수량", "외국계", "프로그램","외인비"]
stock_price = stock_price[use_cols]

stock_price.head()

In [ ]:
stock_price.shape

In [ ]:
# 불필요한 부호 제거
stock_price.loc[:,"시가":"종가"] = stock_price.loc[:,"시가":"종가"].apply(lambda x: abs(x))
stock_price.loc[:,"개인":"프로그램"] = stock_price.loc[:,"개인":"프로그램"].astype(str)
stock_price.loc[:, "개인":"프로그램"] = stock_price.loc[:, "개인":"프로그램"].applymap(lambda x: int(re.sub(r"[^0-9]", "", x))\
    if x[0] == '+' else int(re.sub(r"[^0-9]", "", x)) * -1)

# str타입을 datetime타입으로 변경
stock_price["날짜"] = stock_price["날짜"].apply(lambda x: datetime.datetime.strptime(str(x), "%Y%m%d"))

# 종속변수를 위한 컬럼 추가 및 컬럼순서 변경
n = 10 # 최대 n일 뒤의 종가를 예측하고자 함.
for i in range(1, n+1):
    stock_price[f"{i}일 뒤 종가"] = stock_price["종가"].shift(i)

col1=stock_price.columns[1:-n].to_list()
col2=stock_price.columns[-n:].to_list()
new_col = stock_price.columns[0:1].to_list() + col2 + col1
stock_price=stock_price[new_col]

# 쉬프트 이전 제거.
stock_price = stock_price[n:]

###########################
# 종속변수 분류 정의

# 종가기준 p% 이상이면 2
# 종가기준 0% 이상이면 1
# 종가기준 마이너스면 0
profit = 0.03
for i in range(1, n+1):
    stock_price[f"{i}일 뒤 종가"] = stock_price.apply(lambda x: 2 \
        if ((x[f"{i}일 뒤 종가"]-x["종가"])/x["종가"] > profit ) else 1 \
            if x[f"{i}일 뒤 종가"] > x["종가"] else 0, axis=1)

stock_price.reset_index(drop=True, inplace=True)
stock_price.head()

In [ ]:
stock_price.shape

In [ ]:
file_name = "../data/preprocessed_data/samsung_price.csv"
stock_price.to_csv(file_name, encoding='cp949')